<a href="https://colab.research.google.com/github/RenataTNT/geekbrains_tasks/blob/master/Renata_Irnazarova_Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.
Напишите текстом как минимум 6 функций распределения и соответствующие им функции правдоподобия.
Напишите, какие по вашему мнению целевые переменные из бизнеса (лучше - из страхового) соответствуют этим распределениям (по 2 примера на каждое).
Пример: биномиальное распределение для задачи моделирования пролонгации (продлил клиент полис на следующий период или нет).

2.
На основе датасета из ноутбука проведите анализ данных и постройте зависимости частоты и среднего убытка от какого-либо параметра (за исключением уже выполненных в ноутбуке).

1.1 Равномерное распределение: у клиента через 3 дня заканчивается срок действия полиса. Вероятность того, что он оформит новый полис в конкретный день из последующих 3х дней равномерно распределена. Может использоваться в задаче краткосрочного планирования продаж.

1.2 Нормальное распределение: стоимость полисов ОСАГО, проданных за год, если анализировать их количество в зависимости от диапазона стоимости. Может использоваться в задачах прогнозирования цен на будущий период

1.3 Экспоненциальное распределение: мне кажется, что ему может починяться количество застрахованных по каско авто в зависимости от их возрата, т.к. скорее всего новые машины чаше страхуют. Может использоваться в оценке целесообразности открытия оффиса продаж рядом с диллерским центром.

1.4.  Биномиальное распределение: вероятность продать 10 полисов в зависимости от количества звонков клиентам, у которых старых полис заканчивается. Может использоваться в постановке таргетов колл-центру

1.5 F-распределение: мне кажется ему может подчиняться вероятность страхового случая от стажа. Т.к. в первые месяцы еще аккуратно водят, потом начинают лихачить до 1-ой аварии :) Потом водят более аккуратно за счет опыта. Может использоваться в установке коэфиициента за стаж

1.6 Симметричное пункту 1.5 (не знаю, как называется) может быть распределение частоты угонов в зависимости от рыночной стоимости авто (мне так кажется). Чем дороже авто, тем более велик соблазн его угона, но при этом самые дорпогие эксклюзивные модели вряд ли кто-то захочет угонять и быстро быть найденным 

д/з 2

In [0]:
# Загрузка библиотек

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Датасет: freMPL (French Motor Personal Line datasets)

Источник данных: http://cas.uqam.ca/

Продукт: КАСКО

Набор из 10 датасетов частного французского автостраховщика. Каждый датасет содержит характеристики риска, суммы величин страховых требований и истории страховых исков по около 30 000 страховых полисов за 2004 год.

In [0]:
# Загрузка набора данных в pandas DataFrame

df = pd.read_csv('freMPL-R.csv', low_memory=False)

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343080 entries, 0 to 343079
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           343080 non-null  float64
 1   LicAge             343080 non-null  int64  
 2   RecordBeg          343080 non-null  object 
 3   RecordEnd          181115 non-null  object 
 4   VehAge             177880 non-null  object 
 5   Gender             343080 non-null  object 
 6   MariStat           343080 non-null  object 
 7   SocioCateg         343080 non-null  object 
 8   VehUsage           343080 non-null  object 
 9   DrivAge            343080 non-null  int64  
 10  HasKmLimit         343080 non-null  int64  
 11  BonusMalus         343080 non-null  int64  
 12  VehBody            145780 non-null  object 
 13  VehPrice           145780 non-null  object 
 14  VehEngine          145780 non-null  object 
 15  VehEnergy          145780 non-null  object 
 16  Ve

In [145]:
df.head()

,Exposure,LicAge,RecordBeg,RecordEnd,VehAge,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,BonusMalus,VehBody,VehPrice,VehEngine,VehEnergy,VehMaxSpeed,VehClass,ClaimAmount,RiskVar,Garage,ClaimInd,Dataset,DeducType,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea
0,0.583,366,2004-06-01,NaN,2,Female,Other,CSP1,Professional,55,0,72,sedan,D,injection,regular,160-170 km/h,B,0.000000,15.0,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.200,187,2004-10-19,NaN,0,Male,Alone,CSP55,Private+trip to office,34,0,80,microvan,K,direct injection overpowered,diesel,170-180 km/h,M1,0.000000,20.0,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.083,169,2004-07-16,2004-08-16,1,Female,Other,CSP1,Professional,33,0,63,other microvan,L,direct injection overpowered,diesel,170-180 km/h,M1,0.000000,17.0,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.375,170,2004-08-16,NaN,1,Female,Other,CSP1,Professional,34,0,63,other microvan,L,direct injection overpowered,diesel,170-180 km/h,M1,0.000000,17.0,Private garage,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.500,224,2004-01-01,2004-07-01,3,Male,Other,CSP47,Professional,53,0,72,van,L,direct injection overpowered,diesel,140-150 km/h,0,1418.610272,19.0,None,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Проверка качества заполнения данных

Поскольку freMPL состоит из 10 отдельных датасетов, каждый из которых имеет различные переменные, есть смысл заранее посмотреть, какие датасеты стоит использовать, чтобы затем не возникало проблем с большим количеством пропущенных данных.

In [146]:
# Смотрим, какие факторы пропущены в каждом из датасетов

dct = {}
for i in range(1,11):
    _x = df.loc[df.Dataset == i].notnull().sum()
    dct[i] = list(_x[_x == 0].index)

print('Dataset  Missing Variables')
for x in range(1,11):
    print(x,'\t',dct[x])

Dataset  Missing Variables
1 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
2 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
3 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
4 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
5 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
6 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
7 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
8 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass

Наборы данных 1-4 не содержат информации о количестве страховых требований, а 5-10 не содержат информации по характеристикам транспортного средства. Тем не менее, наборы 5-9 имеют одинаковые пропущенные факторы, имеет смысл попробовать их объединить.

In [147]:
# Объединяем наборы данных 5-9, удаляем пустые столбцы, удаляем дубликаты

df59 = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df59 = df59.drop(['Dataset'], axis=1)
df59 = df59.dropna(axis=1, how='all')
print('With duplicates\t\t', len(df59))
df59 = df59.drop_duplicates()
print('Without duplicates\t', len(df59))

With duplicates		 165200
Without duplicates	 115155


Внутри каждого набора данных имеется около 10 000 наблюдений дубликатов. Все дубликаты были удалены.

In [148]:
df59.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115155 entries, 145780 to 310979
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 1

In [0]:
df = df59

In [0]:
# Вспомогательный столбец для суммирования числа полисов

df['PolicyCount'] = 1

In [0]:
# Вспомогательный столбец для суммирования числа полисов с убытками

df['ClaimCount'] = df['ClaimAmount'] > 0

In [0]:
# Вспомогательный столбец для суммирования числа полисов без убытков

df['NoClaimCount'] = df.PolicyCount - df.ClaimCount

In [153]:
df[['PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].head()

,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
145780,1,0.0,False,1
145781,1,0.0,False,1
145782,1,0.0,False,1
145783,1,0.0,False,1
145784,1,0.0,False,1


##Основные зависимости

Смотрим зависимость от целей использования автомобиля

In [0]:
df_group_VehUsage = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['VehUsage']).sum()
df_group_VehUsage = df_group_VehUsage.reset_index()

In [0]:
df_group_VehUsage['Freq'] = df_group_VehUsage.ClaimCount / df_group_VehUsage.Exposure
df_group_VehUsage['AvgClaim'] = df_group_VehUsage.ClaimAmount / df_group_VehUsage.ClaimCount

In [156]:
df_group_VehUsage.head()

,VehUsage,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,Private,17385.293,38839,5.630104e+06,3066.0,35773,0.176356,1836.302806
1,Private+trip to office,25945.001,59834,1.232871e+07,5812.0,54022,0.224012,2121.250126
2,Professional,6574.757,14302,3.658391e+06,1718.0,12584,0.261302,2129.447356
3,Professional run,977.100,2180,4.318442e+05,273.0,1907,0.279398,1581.846871


In [157]:
fig = px.bar(df_group_VehUsage, x='VehUsage', y='Freq', title='Зависимость частоты убытков от цели использования авто')
fig.show()

Видно, что частота страховых случаев для корпоративного и спортивного транспорта выше, чем для личного

In [158]:
fig = px.bar(df_group_VehUsage, x='VehUsage', y='AvgClaim', title='Зависимость средних убытков от цели использования авто')
fig.show()

Несмотря на высокую частоту страховых случаев у спортивного транспорта, средние убытки по ним ниже. Вероятно, это связано с тем, что их реже угоняют и по ним не возмещают полную страховую стоимость авто. Попробуем проверить

ClaimNbResp Number of responsible claims in the 4 preceding years.
ClaimNbNonResp Number of non-responsible claims in the 4 preceding years.
ClaimNbParking Number of parking claims in the 4 preceding years.
ClaimNbFireTheft Number of fire-theft claims in the 4 preceding years.
ClaimNbWindscreen Number of windscreen claims in the 4 preceding years.

In [159]:
df_group_VehUsage2 = df[['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen']].groupby(df['VehUsage']).sum()
df_group_VehUsage2 = df_group_VehUsage2.reset_index()
df_group_VehUsage2

,VehUsage,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen
0,Private,9679.0,9824.0,3304.0,2315.0,11167.0
1,Private+trip to office,14729.0,19917.0,4192.0,5156.0,23032.0
2,Professional,4821.0,5635.0,1419.0,1225.0,6842.0
3,Professional run,768.0,1002.0,226.0,191.0,1279.0


In [160]:
df_group_VehUsage2['four_years_cases']=df_group_VehUsage2.sum(axis=1)
df_group_VehUsage2

,VehUsage,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,four_years_cases
0,Private,9679.0,9824.0,3304.0,2315.0,11167.0,36289.0
1,Private+trip to office,14729.0,19917.0,4192.0,5156.0,23032.0,67026.0
2,Professional,4821.0,5635.0,1419.0,1225.0,6842.0,19942.0
3,Professional run,768.0,1002.0,226.0,191.0,1279.0,3466.0


In [161]:
df_group_VehUsage2['ClaimNbResp']=df_group_VehUsage2.ClaimNbResp/df_group_VehUsage2.four_years_cases
df_group_VehUsage2['ClaimNbNonResp']=df_group_VehUsage2.ClaimNbNonResp/df_group_VehUsage2.four_years_cases
df_group_VehUsage2['ClaimNbParking']=df_group_VehUsage2.ClaimNbParking/df_group_VehUsage2.four_years_cases
df_group_VehUsage2['ClaimNbFireTheft']=df_group_VehUsage2.ClaimNbFireTheft/df_group_VehUsage2.four_years_cases
df_group_VehUsage2['ClaimNbWindscreen']=df_group_VehUsage2.ClaimNbWindscreen/df_group_VehUsage2.four_years_cases
df_group_VehUsage2

,VehUsage,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,four_years_cases
0,Private,0.266720,0.270716,0.091047,0.063793,0.307724,36289.0
1,Private+trip to office,0.219751,0.297153,0.062543,0.076925,0.343628,67026.0
2,Professional,0.241751,0.282569,0.071156,0.061428,0.343095,19942.0
3,Professional run,0.221581,0.289094,0.065205,0.055107,0.369013,3466.0


In [168]:
#df_group_VehUsage2[['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen']].plot(kind='bar', x=)
fig = px.bar(df_group_VehUsage2, x='VehUsage', y='ClaimNbFireTheft', title='Зависимость количества случаев "угона/полной гибели" от цели использования авто')
fig.show()

действительно, доля случаев, связанных с угонами у спортивных тачек меньше :)